In [ ]:
import os
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow import keras
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

class_names = ['0', '1', '2', '3', 
               '4', '5', '6', '7', 
               '8', '9', 'A', 'B', 
               'C', 'D', 'E', 'F', 
               'G', 'H', 'K', 'L', 
               'M', 'N', 'O', 'P', 
               'R', 'S', 'T', 'U', 
               'V', 'X', 'Y', 'Z']

datagen = ImageDataGenerator(
    rescale=1.0/255, 
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='training',
    class_mode='sparse'
)

validation_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='validation',
    class_mode='sparse'
)

In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(128, (5,5), activation='elu', 
#                            padding='same',
#                            input_shape=(28, 12, 3)),
#     tf.keras.layers.Conv2D(128, (5,5), activation='elu',
#                            padding='same'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(256, (3,3), activation='elu', 
#                            padding='same'),
#     tf.keras.layers.Conv2D(256, (3,3), activation='elu',
#                            padding='same'),
#     tf.keras.layers.Conv2D(128, (3,3), activation='elu',
#                            padding='same'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='elu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(128, activation='elu'),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(32, activation='softmax')
# ])
model = tf.keras.models.load_model('../Model/Model2')

In [ ]:
model.summary()

In [ ]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

In [ ]:
# class stopCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}) :
#         if (logs.get('accuracy') > 0.99) :
#             print('\nReached 99% accuracy so stopping training')
#             self.model.stop_training = True

In [ ]:
# callback = stopCallback()
# model.fit(train_generator, epochs=100, callbacks=[callback])

In [ ]:
# model.evaluate(validation_generator, verbose=1)

In [ ]:
path1 = "../Testing_set/greenparkingchar"
path2 = "../Testing_set/greenparking2char"

all_path = [path1, path2]

destination = "../charLabel/"
trash = "../charLabel/ZTrash"

for path in all_path:
    for file in os.listdir(path) :
        image_mp= mpimg.imread(path + '/' + file) / 255
        image_mp = np.expand_dims(image_mp, axis=0)
        prediction = model.predict(image_mp)
        if (np.max(prediction) < 0.8) :
            if (np.max(prediction) < 0.65) : 
                shutil.move(path + '/' + file, trash)
                continue
        else: 
            shutil.move(path + '/' + file, destination + class_names[np.argmax(prediction)] + '/' + file)

In [ ]:
# model.save('../Model/Model2')